In [1]:
import pandas as pd
import datetime
from datetime import datetime as dt
import re

In [2]:
raw_data = pd.read_csv("../data/processed/json_data_df.csv")
data = raw_data.copy()
df = pd.DataFrame(data)
df.head()

,video_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,...,statistics.favoriteCount,statistics.commentCount,contentDetails.duration,contentDetails.dimension,contentDetails.definition,contentDetails.caption,contentDetails.licensedContent,contentDetails.projection,contentDetails.regionRestriction.blocked,contentDetails.contentRating.ytRating
0,uGtc9Bu9Txk,2020-12-31T23:00:10Z,UCArk93C2pbOvkv6jWz-3kAg,🍾 FORD KIERNAN HOGMANAY SPECIAL! | Open Goal,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/uGtc9Bu9Txk/default.jpg,120,90,https://i.ytimg.com/vi/uGtc9Bu9Txk/mqdefault.jpg,320,...,0,112,PT1H21M55S,2d,hd,False,True,rectangular,NaN,NaN
1,vAGmV-mRRT0,2020-12-24T07:00:08Z,UCArk93C2pbOvkv6jWz-3kAg,CHRISTMAS EVE SPECIAL | Right in the Coupon w/...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/vAGmV-mRRT0/default.jpg,120,90,https://i.ytimg.com/vi/vAGmV-mRRT0/mqdefault.jpg,320,...,0,30,PT44M31S,2d,hd,False,True,rectangular,NaN,NaN
2,m54LPZSlStQ,2020-12-22T17:00:20Z,UCArk93C2pbOvkv6jWz-3kAg,EPIC CUP FINAL REVIEW | Keeping the Ball on th...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/m54LPZSlStQ/default.jpg,120,90,https://i.ytimg.com/vi/m54LPZSlStQ/mqdefault.jpg,320,...,0,63,PT1H6M12S,2d,hd,False,True,rectangular,NaN,NaN
3,1RcFj1Sfs58,2020-12-21T17:00:03Z,UCArk93C2pbOvkv6jWz-3kAg,JAMIE CARRAGHER | Open Goal Meets...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/1RcFj1Sfs58/default.jpg,120,90,https://i.ytimg.com/vi/1RcFj1Sfs58/mqdefault.jpg,320,...,0,661,PT2H22M14S,2d,hd,False,True,rectangular,NaN,NaN
4,MaJhyEr6csQ,2020-12-17T17:25:28Z,UCArk93C2pbOvkv6jWz-3kAg,CELTIC v HEARTS & ST MIRREN END RANGERS UNBEAT...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/MaJhyEr6csQ/default.jpg,120,90,https://i.ytimg.com/vi/MaJhyEr6csQ/mqdefault.jpg,320,...,0,49,PT45M40S,2d,hd,False,True,rectangular,NaN,NaN


# Remove prefixes

In [3]:
def remove_prefixes(column_name):
    """Removes the prefixes e.g., 'snippet' and 'statistics' from the column titles.
    
    
    Parameters
    -----------
    column_name (str):
        A string representing a column name.
        
    Returns
    --------
    column_name (str):
        Returns the input without the prefixes specified in the prefixes_and_replacement
        dictionary.
        
    Example Use
    ------------
    df.columns = df.columns.map(remove_prefixes)
    
    Notes
    ------
    
    """
    # define the prefixes we want to remove and what we'll replace them with (nothing)
    prefixes_and_replacement = {"snippet.": "", 
                               "statistics.": "",
                               "contentDetails.": ""}
    
    for prefix, replacement in prefixes_and_replacement.items():
        if column_name.startswith(prefix):
            column_name = column_name.replace(prefix, replacement)
            
    return column_name

In [4]:
df.columns = df.columns.map(remove_prefixes)
df.columns

Index(['video_id', 'publishedAt', 'channelId', 'title', 'description',
       'thumbnails.default.url', 'thumbnails.default.width',
       'thumbnails.default.height', 'thumbnails.medium.url',
       'thumbnails.medium.width', 'thumbnails.medium.height',
       'thumbnails.high.url', 'thumbnails.high.width',
       'thumbnails.high.height', 'thumbnails.standard.url',
       'thumbnails.standard.width', 'thumbnails.standard.height',
       'thumbnails.maxres.url', 'thumbnails.maxres.width',
       'thumbnails.maxres.height', 'channelTitle', 'tags', 'categoryId',
       'liveBroadcastContent', 'localized.title', 'localized.description',
       'defaultAudioLanguage', 'viewCount', 'likeCount', 'favoriteCount',
       'commentCount', 'duration', 'dimension', 'definition', 'caption',
       'licensedContent', 'projection', 'regionRestriction.blocked',
       'contentRating.ytRating'],
      dtype='object')

# Transform duration

In [5]:
def duration_to_hhmmss(duration):
    """Takes in a string of YouTube's API duration 'response' and uses a regex to return two values:
        - the duration in sconds as a timedelta object
        - the duration in hh:mm:ss format.
    
    Parameters
    ----------
    duration (str):
        YouTube's API duration response which provides the duration of a 
        video as a string with the format "PT1H21M36S" for 1 hour, 21 minutes
        and 36 seconds.
        
        
    Returns
    --------
    duration_timedelta (timedelta):
        The input transformed into a timedelta objct which provids the duration in seconds.
        
    duration_string (str):
        The input transformed into a string with format: "hh:mm:ss".
        
        
    Example Use
    ------------
    Extracting duration of YouTube video:
        - Create a column that can compare the durations of the videos with:
            
                df['duration_seconds'] = df['duration'].apply(lambda x: duration_to_hhmmss(x)[0])
                
        - Create a quickly interpretable duration column with:
        
                df['duration_string'] = df['duration'].apply(lambda x: duration_to_hhmmss(x)[1])
    
    """
    
    # set duration pattern to capture the YouTube API response which gives duration in format PT1H52M21S
    # for 1 hour, 52 minutes, 21 seconds
    duration_pattern = r"[A-Za-z]+(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?"
    
    # use regex to compile the pattern
    duration_regex = re.compile(duration_pattern)
    
    # match the pattern and store the groups returned in a variable
    duration_groups = duration_regex.match(duration)
    
    # store each group in a variable
    full_response, hours, minutes, seconds = duration_groups[0], duration_groups[1], duration_groups[2], duration_groups[3]
    
    # ensure we have a non-Null value for hours, minutes, and seconds, and zero pad them
    if hours != None:
        if len(hours) == 1:
            hours = '0' + hours
            
    elif hours == None:
        hours = '00'
            
    if minutes != None:
        if len(minutes) == 1:
            minutes = '0' + minutes
                       
    elif minutes == None:
        minutes = '00'
    
    if seconds != None:
        if len(seconds) == 1:
            seconds = '0' + seconds
                          
    elif seconds == None:
        seconds = '00'
     
    
    # create datetime object
    duration_string = f"{hours}:{minutes}:{seconds}"
    
#     duration_formatted = dt.strptime(duration_string, "%H:%M:%S")
    
    # use the groups retrieved to create a time object
    duration_timedelta = dt.timedelta(days=0, hours=int(hours), minutes=int(minutes), seconds=int(seconds))
    
    return duration_timedelta, duration_string

In [6]:
df['duration_timedelta'] = df['duration'].apply(lambda x: duration_to_hhmmss(x)[0])
df['duration_hhmmss'] = df['duration'].apply(lambda x: duration_to_hhmmss(x)[1])
df.head()

,video_id,publishedAt,channelId,title,description,thumbnails.default.url,thumbnails.default.width,thumbnails.default.height,thumbnails.medium.url,thumbnails.medium.width,...,duration,dimension,definition,caption,licensedContent,projection,regionRestriction.blocked,contentRating.ytRating,duration_timedelta,duration_hhmmss
0,uGtc9Bu9Txk,2020-12-31T23:00:10Z,UCArk93C2pbOvkv6jWz-3kAg,🍾 FORD KIERNAN HOGMANAY SPECIAL! | Open Goal,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/uGtc9Bu9Txk/default.jpg,120,90,https://i.ytimg.com/vi/uGtc9Bu9Txk/mqdefault.jpg,320,...,PT1H21M55S,2d,hd,False,True,rectangular,NaN,NaN,0 days 01:21:55,01:21:55
1,vAGmV-mRRT0,2020-12-24T07:00:08Z,UCArk93C2pbOvkv6jWz-3kAg,CHRISTMAS EVE SPECIAL | Right in the Coupon w/...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/vAGmV-mRRT0/default.jpg,120,90,https://i.ytimg.com/vi/vAGmV-mRRT0/mqdefault.jpg,320,...,PT44M31S,2d,hd,False,True,rectangular,NaN,NaN,0 days 00:44:31,00:44:31
2,m54LPZSlStQ,2020-12-22T17:00:20Z,UCArk93C2pbOvkv6jWz-3kAg,EPIC CUP FINAL REVIEW | Keeping the Ball on th...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/m54LPZSlStQ/default.jpg,120,90,https://i.ytimg.com/vi/m54LPZSlStQ/mqdefault.jpg,320,...,PT1H6M12S,2d,hd,False,True,rectangular,NaN,NaN,0 days 01:06:12,01:06:12
3,1RcFj1Sfs58,2020-12-21T17:00:03Z,UCArk93C2pbOvkv6jWz-3kAg,JAMIE CARRAGHER | Open Goal Meets...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/1RcFj1Sfs58/default.jpg,120,90,https://i.ytimg.com/vi/1RcFj1Sfs58/mqdefault.jpg,320,...,PT2H22M14S,2d,hd,False,True,rectangular,NaN,NaN,0 days 02:22:14,02:22:14
4,MaJhyEr6csQ,2020-12-17T17:25:28Z,UCArk93C2pbOvkv6jWz-3kAg,CELTIC v HEARTS & ST MIRREN END RANGERS UNBEAT...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,https://i.ytimg.com/vi/MaJhyEr6csQ/default.jpg,120,90,https://i.ytimg.com/vi/MaJhyEr6csQ/mqdefault.jpg,320,...,PT45M40S,2d,hd,False,True,rectangular,NaN,NaN,0 days 00:45:40,00:45:40


# Drop columns

In [9]:
def drop_columns(df):
    """Drops columns and returns a dataframe.
    
    Parameters
    -----------
    df (dataframe):
        pandas dataframe. 
        
    Returns
    --------
    df_ (dataframe)
        dataframe with columns dropped.
    
    
    
    """
    
    # create a copy of the dataframe
    df_ = df.copy()
    
    # store the columns we want to drop in a list that we'll loop through later
    cols_to_drop = ['thumbnails.default.url', 'thumbnails.default.width',
                    'thumbnails.default.height', 'thumbnails.medium.url',
                    'thumbnails.medium.width', 'thumbnails.medium.height',
                    'thumbnails.high.url', 'thumbnails.high.width',
                    'thumbnails.high.height', 'thumbnails.standard.url',
                    'thumbnails.standard.width', 'thumbnails.standard.height',
                    'thumbnails.maxres.url', 'thumbnails.maxres.width',
                    'thumbnails.maxres.height', 'localized.title', 'localized.description', 
                    'defaultAudioLanguage', 'categoryId', 'liveBroadcastContent', 
                    'dimension', 'duration', 'definition', 'caption','licensedContent', 
                    'projection', 'regionRestriction.blocked', 'contentRating.ytRating']
    

    
    # loop through columns to drop list and drop if the column exists
    for column in cols_to_drop:
        try:
            df_.drop(column, axis=1, inplace=True)
            
        except KeyError as ke:
            print(ke)
            continue
    
    return df_

In [10]:
df = drop_columns(df)
df.head()

"['thumbnails.default.url'] not found in axis"
"['example'] not found in axis"
"['thumbnails.default.width'] not found in axis"
"['thumbnails.default.height'] not found in axis"
"['thumbnails.medium.url'] not found in axis"
"['thumbnails.medium.width'] not found in axis"
"['thumbnails.medium.height'] not found in axis"
"['thumbnails.high.url'] not found in axis"
"['thumbnails.high.width'] not found in axis"
"['thumbnails.high.height'] not found in axis"
"['thumbnails.standard.url'] not found in axis"
"['thumbnails.standard.width'] not found in axis"
"['thumbnails.standard.height'] not found in axis"
"['thumbnails.maxres.url'] not found in axis"
"['thumbnails.maxres.width'] not found in axis"
"['thumbnails.maxres.height'] not found in axis"
"['defaultAudioLanguage'] not found in axis"
"['categoryId'] not found in axis"
"['liveBroadcastContent'] not found in axis"
"['dimension'] not found in axis"
"['duration'] not found in axis"
"['definition'] not found in axis"
"['caption'] not found 

,video_id,publishedAt,channelId,title,description,channelTitle,tags,viewCount,likeCount,favoriteCount,commentCount,duration_timedelta,duration_hhmmss
0,uGtc9Bu9Txk,2020-12-31T23:00:10Z,UCArk93C2pbOvkv6jWz-3kAg,🍾 FORD KIERNAN HOGMANAY SPECIAL! | Open Goal,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'still game', 'jack ...",205682,2286,0,112,0 days 01:21:55,01:21:55
1,vAGmV-mRRT0,2020-12-24T07:00:08Z,UCArk93C2pbOvkv6jWz-3kAg,CHRISTMAS EVE SPECIAL | Right in the Coupon w/...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'betting', 'slaney',...",63357,742,0,30,0 days 00:44:31,00:44:31
2,m54LPZSlStQ,2020-12-22T17:00:20Z,UCArk93C2pbOvkv6jWz-3kAg,EPIC CUP FINAL REVIEW | Keeping the Ball on th...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'celtic vs hearts', ...",85262,976,0,63,0 days 01:06:12,01:06:12
3,1RcFj1Sfs58,2020-12-21T17:00:03Z,UCArk93C2pbOvkv6jWz-3kAg,JAMIE CARRAGHER | Open Goal Meets...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'Jamie Carragher', '...",855101,4966,0,661,0 days 02:22:14,02:22:14
4,MaJhyEr6csQ,2020-12-17T17:25:28Z,UCArk93C2pbOvkv6jWz-3kAg,CELTIC v HEARTS & ST MIRREN END RANGERS UNBEAT...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'celtic v hearts', '...",64383,659,0,49,0 days 00:45:40,00:45:40


In [ ]:
#  define function that splits publishedAt into publishedAtYear, Month and Day

# Export to csv

In [58]:
df.to_csv("../data/processed/json_data_processed.csv", index=False )